In [100]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/titanic/train.csv
/kaggle/input/titanic/test.csv
/kaggle/input/titanic/gender_submission.csv


In [101]:
import pandas as pd
train_df = pd.read_csv('/kaggle/input/titanic/train.csv')
test_df = pd.read_csv('/kaggle/input/titanic/test.csv')
all_df = pd.concat([train_df, test_df], sort=False)
# all_df.info()

In [102]:
age_median = all_df['Age'].median()
all_df['Age'] = all_df['Age'].fillna(age_median)

In [103]:
all_df['FamilySize'] = all_df['SibSp'] + all_df['Parch'] + 1

In [104]:
all_df['Title'] = all_df['Name'].str.extract(' ([A-Za-z]+)\.', expand=False)
title_mapping = {"Mr": 1, "Miss": 2, "Mrs": 3, "Master": 4, "Rare": 5}
all_df['Title'] = all_df['Title'].map(title_mapping)
all_df['Title'] = all_df['Title'].fillna(0)

In [105]:
train_processed_df = all_df[:len(train_df)]
test_processed_df = all_df[len(train_df):]

In [106]:
train_sex_dummies = pd.get_dummies(train_processed_df['Sex'],prefix='Sex')
# train_sex_dummies.head()
train_processed_df = pd.concat([train_processed_df, train_sex_dummies], axis = 1)
train_processed_df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,FamilySize,Title,Sex_female,Sex_male
0,1,0.0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S,2,1.0,False,True
1,2,1.0,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C,2,3.0,True,False
2,3,1.0,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S,1,2.0,True,False
3,4,1.0,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S,2,3.0,True,False
4,5,0.0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S,1,1.0,False,True


In [107]:
features = ['Pclass', 'Age', 'FamilySize', 'Title', 'Sex_female', 'Sex_male']
X_train = train_processed_df[features]
y_train = train_processed_df['Survived']

In [108]:
## from sklearn.ensemble import RandomForestClassifier
## model = RandomForestClassifier(n_estimators=100, max_depth=5, random_state=1)
## model.fit(X_train, y_train)

##overfitting(->bad score)
#from sklearn.tree import DecisionTreeClassifier
#model = DecisionTreeClassifier()
#model.fit(X_train,y_train)

from sklearn.ensemble import RandomForestClassifier
model = RandomForestClassifier(n_estimators=100, max_depth=5, random_state=1)
model.fit(X_train, y_train)

RandomForestClassifier(max_depth=5, random_state=1)

In [109]:
test_sex_dummies = pd.get_dummies(test_processed_df['Sex'], prefix='Sex')
test_processed_df = pd.concat([test_processed_df, test_sex_dummies], axis=1)
X_test = test_processed_df[features]

predictions = model.predict(X_test)

In [110]:
submission = pd.DataFrame({
    "PassengerId": test_df["PassengerId"],
    "Survived": predictions.astype(int) # 念のため整数に変換
})
submission.to_csv('submission_v2.csv', index=False)

print("Successfully created submission_v2.csv!")

Successfully created submission_v2.csv!


In [111]:
from sklearn.model_selection import cross_val_score

model = RandomForestClassifier(n_estimators=100, max_depth=5, random_state=1)

scores = cross_val_score(model, X_train, y_train, cv=4)

print("scores:", scores)
print("mean:", np.mean(scores))
print("std:", np.std(scores))

scores: [0.82959641 0.85201794 0.80717489 0.84684685]
mean: 0.8339090211287521
std: 0.017525928284284197
